In [2]:
import sys
sys.path.append("../")
from monitoring.data_collector import Online_Collector, Batch_Collector
from monitoring.data_drift import Drift_Analysis
import pandas as pd
import time
from azureml.core import Workspace
ws = Workspace.from_config()

kv=ws.get_default_keyvault()


Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azure-core 1.22.1 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azure-core<1.22'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azure-core 1.22.1 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azure-core<1.22'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azure-core 1.22.1 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azure-core<1.22'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline

#### Prerequisite
1. Create a service principal and secret (SP)
2. Provision an ADX cluster and create a DB
3. Add the SP to be contributor of the cluster
4. pip install following packages: pip install --upgrade azure-mgmt-eventhub azure-eventhub azure-mgmt-kusto azure-kusto-ingest azure-kusto-data azure-identity azure-common


#### Provisioning resource


The following code provision target table at ADX, eventhub and ingestion procedure to ingest data automatically once data is available at eventhub

In [21]:

tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
#Application ID
client_id = "af883abf-89dd-4889-bdb3-1ee84f68465e"
#Client Secret, set it at your WS' keyvault with key name same as your client_id
client_secret = kv.get_secret(client_id)
subscription_id = "0e9bace8-7a81-4922-83b5-d995ff706507"

cluster_uri = "https://adx02.westus2.kusto.windows.net" #URL of the ADX Cluster

db_name = "db01"
table_name = "isd_weather_test" #new dataset

sample_pd_data = pd.read_parquet("data/test_data.parquet").head(10)
sample_pd_data['timestamp'] = sample_pd_data['datetime']
sample_pd_data.drop(['datetime'], inplace=True, axis=1)
online_collector = Online_Collector(tenant_id, client_id,client_secret,cluster_uri,db_name,table_name, sample_pd_data)


.create table isd_weather_test (usaf: string, wban: string, latitude: real, longitude: real, elevation: real, windAngle: real, windSpeed: real, temperature: real, seaLvlPressure: real, cloudCoverage: string, presentWeatherIndicator: real, pastWeatherIndicator: real, precipTime: real, precipDepth: real, snowDepth: real, stationName: string, countryOrRegion: string, p_k: string, year: int, day: int, version: real, timestamp: datetime)


In [23]:
time.sleep(120) #It takes about 2 minutes for stream_collect to start working 
online_collector.stream_collect(sample_pd_data)
online_collector.batch_collect(sample_pd_data)

In [24]:
analysis = Drift_Analysis(tenant_id, client_id, client_secret, cluster_uri,db_name)
df = analysis.query(f"""
{table_name}| take(10)
""")
df

,usaf,wban,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,cloudCoverage,...,precipTime,precipDepth,snowDepth,stationName,countryOrRegion,p_k,year,day,version,timestamp
0,999999,53878,35.419,-82.557,641.0,<NA>,<NA>,4.4,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 13 S,US,999999-53878,2008,23,1.0,2008-01-23 00:15:00+00:00
1,999999,53878,35.419,-82.557,641.0,<NA>,<NA>,-1.6,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 13 S,US,999999-53878,2008,21,1.0,2008-01-21 22:55:00+00:00
2,999999,53877,35.495,-82.614,656.0,<NA>,<NA>,-2.3,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 8 SSW,US,999999-53877,2008,22,1.0,2008-01-22 04:55:00+00:00
3,999999,53877,35.495,-82.614,656.0,<NA>,<NA>,1.3,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 8 SSW,US,999999-53877,2008,19,1.0,2008-01-19 09:50:00+00:00
4,999999,53877,35.495,-82.614,656.0,<NA>,<NA>,-2.2,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 8 SSW,US,999999-53877,2008,22,1.0,2008-01-22 03:50:00+00:00
5,999999,53877,35.495,-82.614,656.0,<NA>,<NA>,-0.9,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 8 SSW,US,999999-53877,2008,21,1.0,2008-01-21 22:30:00+00:00
6,999999,53878,35.419,-82.557,641.0,<NA>,0.3,1.0,<NA>,,...,1.0,0.0,<NA>,ASHEVILLE 13 S,US,999999-53878,2008,6,1.0,2008-01-06 13:00:00+00:00
7,999999,53878,35.419,-82.557,641.0,<NA>,<NA>,-9.5,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 13 S,US,999999-53878,2008,25,1.0,2008-01-25 06:35:00+00:00
8,999999,53878,35.419,-82.557,641.0,<NA>,<NA>,-4.2,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 13 S,US,999999-53878,2008,16,1.0,2008-01-16 06:20:00+00:00
9,999999,53878,35.419,-82.557,641.0,<NA>,<NA>,4.0,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 13 S,US,999999-53878,2008,28,1.0,2008-01-28 14:50:00+00:00


In [25]:
#Batch collector to Blob storage for big file
ds = ws.get_default_datastore()
batch_collector = Batch_Collector(ds, "test_folder")
batch_collector.collect("config.json")

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading config.json
Uploaded config.json, 1 files out of an estimated total of 1
Uploaded 1 files


{'filename': 'config.json',
 'datastore': 'workspaceblobstore',
 'folder': 'test_folder'}